True

In [0]:
%fs
ls "/mnt/tokyoolympic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw-data/,raw-data/,0,1744636157000
dbfs:/mnt/tokyoolympic/transform-data/,transform-data/,0,1744636172000


In [0]:
athletes = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/coaches.csv")
events = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/events.csv")
medallists = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/medallists.csv")
medals_total = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/medals_total.csv")
medals = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/medals.csv")
nocs = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/nocs.csv")
schedules = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/schedules.csv")
schedules_preliminary = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/schedules_preliminary.csv")
teams = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/teams.csv")
techofficials = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/technical_officials.csv")
torch_route = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/torch_route.csv")
venues = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/venues.csv")



In [0]:
#athletes.show()
#coaches.show()
#events.show()
#medallists.show()
#medals_total.show()
#medals.show()
#nocs.show()
#schedules.show()        
#schedules_preliminary.show()
#teams.show()
#techofficials.show()
#torch_route.show()
#venues.show()
display(coaches)



com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
from pyspark.sql.functions import to_date, col, when, trim, regexp_replace
#from pyspark.sql.types import IntegerType, BooleanType, StringType, FloatType, DateType

# Step 1: Clean the columns, stripping unwanted characters and trimming
athletes_cleaned = athletes.withColumn("birth_date", trim(col("birth_date"))) \
    .withColumn("birth_place", trim(col("birth_place"))) \
    .withColumn("events", regexp_replace(col("events"), r'[\[\]"\']', ''))

# Step 2: Attempt to convert 'birth_date' to proper date type
athletes_cleaned = athletes_cleaned.withColumn("birth_date_clean", to_date("birth_date", "yyyy-MM-dd"))

# Step 3: Handle possible column shifts (if birth_date is invalid but birth_place is a valid date)
athletes_cleaned = athletes_cleaned.withColumn(
    "final_birth_date",
    when(col("birth_date_clean").isNotNull(), col("birth_date_clean")) 
     .when(to_date(col("birth_place"), "yyyy-MM-dd").isNotNull(), to_date(col("birth_place"), "yyyy-MM-dd"))
    .otherwise(None)
).withColumn(
    "final_birth_place",
    when(to_date(col("birth_place"), "yyyy-MM-dd").isNotNull(), None)
    .otherwise(col("birth_place"))
)

# Step 4: Final cleanup and applying the new values
athletes_cleaned = athletes_cleaned.withColumn("birth_date", col("final_birth_date")) \
    .withColumn("birth_place", col("final_birth_place")) \
    .drop("birth_date_clean", "final_birth_date", "final_birth_place")


# Step 6: View cleaned data and print schema to verify types
athletes_cleaned.printSchema()
athletes_cleaned.show()


In [0]:
from pyspark.sql.functions import count, col

# Step 1: Total number of coaches per country
coaches_cleaned = coaches.groupBy("country").count().withColumnRenamed("count", "total_coaches")

# Step 2: Filter head coaches
coaches_filter = coaches.filter(col("function") == "Head Coach")

# Step 3: Pivot gender counts per country
groupby_gender_country = coaches.groupBy("country").pivot("gender").agg(count("*"))

# Step 4: Join total coaches and gender counts
coaches_combined = coaches_cleaned.join(groupby_gender_country, on="country", how="left")

# Step 5: Join head coach details
# If multiple head coaches exist per country, this will duplicate rows. 
# If you want only 1 row per country, consider using groupBy or dropDuplicates.
coaches_final = coaches_combined.join(coaches_filter, on="country", how="left")

# Step 6: Display final result
display(coaches_final)


In [0]:
from pyspark.sql.functions import col, count

# Step 1: Filter by medal type
gold_df = medals.filter(col("medal_type") == "Gold Medal")
silver_df = medals.filter(col("medal_type") == "Silver Medal")
bronze_df = medals.filter(col("medal_type") == "Bronze Medal")

# Step 2: Count medals per country
gold_count = gold_df.groupBy("country").agg(count("*").alias("gold_count"))
silver_count = silver_df.groupBy("country").agg(count("*").alias("silver_count"))
bronze_count = bronze_df.groupBy("country").agg(count("*").alias("bronze_count"))

# Step 3: Join counts back to original DataFrame
medals_with_counts = medals \
    .join(gold_count, on="country", how="left") \
    .join(silver_count, on="country", how="left") \
    .join(bronze_count, on="country", how="left")

# Step 4: Fill nulls with 0 (optional, for countries with no medals of a type)
medals_with_counts = medals_with_counts.fillna({"gold_count": 0, "silver_count": 0, "bronze_count": 0})

# Step 5: Show the final result
display(medals_with_counts)


In [0]:
from pyspark.sql.functions import trim, col

events_cleaned = events.withColumn("event", trim(col("event"))) \
                       .withColumn("tag", trim(col("tag"))) \
                       .withColumn("sport", trim(col("sport"))) \
                       .withColumn("sport_code", trim(col("sport_code"))) \
                       .withColumn("sport_url", trim(col("sport_url")))
from pyspark.sql.functions import split

events_cleaned = events_cleaned.withColumn("gender", split(col("event"), " ")[0]) \
                               .withColumn("event_type", split(col("event"), " ", 2)[1])

display(events_cleaned)




In [0]:
from pyspark.sql.functions import count, when, col, split, desc

# Step 1: Aggregate medal counts per gender
medal_counts = medallists.groupBy("gender").agg(
    count(when(col("medal_type") == "Gold Medal", 1)).alias("Gold_Medals"),
    count(when(col("medal_type") == "Silver Medal", 1)).alias("Silver_Medals"),
    count(when(col("medal_type") == "Bronze Medal", 1)).alias("Bronze_Medals")
)

# Step 2: Add total medals
medal_counts = medal_counts.withColumn(
    "total_medals", col("Gold_Medals") + col("Silver_Medals") + col("Bronze_Medals")
)

# Step 3: Directly update team and team_gender in original data
medals_team = medallists.withColumn(
    "team", when(col("team").isNull(), split(col("event"), " ", 2).getItem(1)).otherwise(col("team"))
).withColumn(
    "team_gender", when(col("team_gender").isNull(), split(col("event"), " ", 2).getItem(0)).otherwise(col("team_gender"))
)

# Step 4: Join medal counts with updated medallist data
medallists_with_totals = medals_team.join(medal_counts, on="gender", how="left")

# Step 5: Display result
display(medallists_with_totals.orderBy(desc("total_medals")))


In [0]:
display(medals_total)

In [0]:
display(nocs)

In [0]:
display(schedules)

In [0]:
from pyspark.sql.functions import col



# Step 1: Fill null values in 'medal' column with 0
medals_filled = schedules_preliminary.fillna({"medal": 0})

# Step 2: Rename columns to make them more readable
column_rename = medals_filled.withColumnRenamed("date_start_utc", "start_time_utc") \
               .withColumnRenamed("date_end_utc", "end_time_utc")
medals_filled_unique = medals_filled.dropDuplicates(["sport"])
# Step 3: Join the DataFrames on the 'sport' column (or whichever column is the correct common column)
final_schedule = column_rename.join(medals_filled_unique, on="sport", how="left")

# Step 4: Show the result
display(final_schedule)


In [0]:
from pyspark.sql.functions import when, col

# Step 1: Get unique coach names per country
coach_country = coaches.select("country_code", "name").dropDuplicates(["country_code"])

# Step 2: Join with teams on country_code
teams_with_coaches = teams.join(coach_country, on="country_code", how="left")

# Step 3: Fill 'coaches' where it's null
teams_filled = teams_with_coaches.withColumn(
    "coaches",
    when(col("coaches").isNull(), col("name")).otherwise(col("coaches"))
).drop("name")

# Step 4: View the result
display(teams_filled)


In [0]:
from pyspark.sql.functions import col, count
from pyspark.sql.window import Window

# Step 1: Filter judges
judges_df = techofficials.filter(col("function") == "Judge")

# Step 2: Create a window partitioned by gender
gender_window = Window.partitionBy("gender")

# Step 3: Add a new column with gender count
judges_with_gender_count = judges_df.withColumn("gender_count", count("*").over(gender_window))

# Step 4: Display the result
display(judges_with_gender_count)


In [0]:
display(torch_route)

In [0]:
display(venues)

In [0]:
athletes_cleaned.repartition(1).write.mode("overwrite").option("header", True).csv("/mnt/tokyoolymic/transform-data/athletes_cleaned.csv")

coaches_final.write.option("header", True).csv("/mnt/tokyoolympic/transform-data/coaches_final.csv")
medals_with_counts.write.option("header", True).csv("/mnt/tokyoolympic/transform-data/medals_with_counts.csv")
events_cleaned.write.option("header", True).csv("/mnt/tokyoolympic/transform-data/events_cleaned.csv")
medallists_with_totals.write.option("header", True).csv("/mnt/tokyoolympic/transform-data/medallists_with_totals.csv")
medals_total.write.option("header", True).csv("/mnt/tokyoolympic/transform-data/medals_total.csv")
nocs.write.option("header", True).csv("/mnt/tokyoolympic/transform-data/nocs.csv")
schedules.write.option("header", True).csv("/mnt/tokyoolympic/transform-data/schedules.csv")
final_schedule.write.option("header", True).csv("/mnt/tokyoolympic/transform-data/final_schedule.csv")
teams_filled.write.option("header", True).csv("/mnt/tokyoolympic/transform-data/teams_filled.csv")
judges_with_gender_count.write.option("header", True).csv("/mnt/tokyoolympic/transform-data/judges_with_gender_count.csv")
venues.write.option("header", True).csv("/mnt/tokyoolympic/transform-data/venues.csv")
torch_route.write.option("header", True).csv("/mnt/tokyoolympic/transform-data/torch_route.csv")